 # Jensen Lab

 Author: Moshe Silverstein <br/>
 Date: 7-17 <br/>
 Data Sources:
 * http://tissues.jensenlab.org/Search
 * http://diseases.jensenlab.org/Search
 * http://compartments.jensenlab.org/Search

 Reviewer: Charles Dai <br>
 Updated: 6-20

In [ ]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [ ]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [ ]:
%load_ext autoreload
%autoreload 2

 ### Notebook Information

In [ ]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

 # Initialization

 ### Select Jensen Data

In [ ]:
%%appyter code_eval

{% set data = ChoiceField(
    name='group',
    label='Group',
    choices={
        'Compartments': "'Compartments'",
        'Disease': "'Disease'",
        'Tissue': "'Tissue'"
    },
    default='Compartments',
    section='data'
) %}

 ### Load Mapping Dictionaries

In [ ]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

 ### Output Path

In [ ]:
%%appyter code_exec

output_name = 'jensen_' + {{data}}.lower()

path = 'Output/Jensen-' + {{data}}
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from any of the Jensen Databases',
) %}

 # Load Data

In [ ]:
%%appyter code_exec

file_list = []
file_list.append({{FileField(constraint='(.*\.tsv$)?',
    name='file1', label='Source 1', 
    default='Input/Jensen/human_disease_textmining_filtered.tsv',
    section='data') 
}})
file_list.append({{FileField(constraint='(.*\.tsv$)?', 
    name='file2', label='Source 2', 
    default='Input/Jensen/human_disease_textmining_filtered.tsv',
    section='data') 
}})
file_list.append({{FileField(constraint='(.*\.tsv$)?', 
    name='file3', label='Source 3', 
    default='Input/Jensen/human_disease_textmining_filtered.tsv',
    section='data') 
}})

In [ ]:
%%appyter code_exec

df_list = []
for f in file_list:
    if f:
        df_list.append(pd.read_csv(f, sep='\t', 
                        usecols=[1, 3], names=['GeneSym', {{data}}], 
                        index_col=0))
df = pd.concat(df_list)

In [ ]:
df.head()

In [ ]:
df.shape

 # Filter Data

 ## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
df = uf.mapgenesymbols(df, symbol_lookup)
df.shape

 # Analyze Data

 ## Create Binary Matrix

In [ ]:
binary_matrix = uf.createBinaryMatrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.saveData(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

 ## Create Gene List

In [ ]:
gene_list = uf.createGeneList(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.saveData(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

 ## Create Attribute List

In [ ]:
attribute_list = uf.createAttributeList(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.saveData(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

 ## Create Gene and Attribute Set Libraries

In [ ]:
uf.createUpGeneSetLib(binary_matrix, path, output_name + '_gene_up_set')

In [ ]:
uf.createUpAttributeSetLib(binary_matrix, path, 
                           output_name + '_attribute_up_set')

 ## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.createSimilarityMatrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.saveData(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

 ## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.createSimilarityMatrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.saveData(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

 ## Create Gene-Attribute Edge List

In [ ]:
uf.createGeneAttributeEdgeList(binary_matrix, attribute_list, gene_list, 
                               path, output_name + '_gene_attribute_edge_list')

 # Create Downloadable Save File

In [ ]:
uf.createArchive(path)

 ### Link to download output files: [click here](./output_archive.zip)